# Control flow

## Boolean types

 Booleans are special data types that can have two values, `true` or `false`. 
 They are essential for conducting control flow.



Boolean operators (copied from [here](https://docs.julialang.org/en/v1/manual/mathematical-operations/#Boolean-Operators))


|Expression|	Name|
|--|--|
|`!x`|	negation|
|`x && y`|	short-circuiting `and`|
|`x \|\| y`|	short-circuiting `or`|

Numeric comparisons (copied partially from [here](https://docs.julialang.org/en/v1/manual/mathematical-operations/#Numeric-Comparisons))
|Operator|	Name|
|--|--|
|`==`	|equality|
|`!=`, `≠`	|inequality|
|`<`	|less than|
|`<=`, `≤`|	less than or equal to|
|`>`|	greater than|
|`>=`, `≥`|	greater than or equal to|
|`≈`, `isapprox` | approximate equality with _relative_ tolerance error `atol`|
|`===` | programmatically identical equality comparison |

Most operators work exactly the same as in other languages.

In [1]:
2 == 2

true

In [2]:
4>5

false

When I was learning coding in `Julia`, two operators were new to me: `≈` and `===`.

Operator `≈` compares two  different objects given the tolerance error `atol`:

In [3]:
≈(2000, 2000.1, atol=.5)

true

In [4]:
≈(2000, 2000.1; atol=.0001)

false

Operator `===` checks whether two objects are programmatically identical. In case of arrays or dictionaries, it means that we check whether two objects refer to the same memory slots. Illustrative example below:

In [5]:
x = [1,2]
y = x
z = deepcopy(x)

println("x==y: $(x==y)") #both objects have the same values
println("x==z: $(x==z)") #both objects have the same values
println("x===y: $(x===y)") #both objects refer to the same memory
println("x===z: $(x===z)") #both objects refer to different memory slots (even though they may have identical values)

x==y: true
x==z: true
x===y: true
x===z: false


The mentioned operators can be broadcasted. As a result we get arrays:

In [6]:
(1:1:3) .== (3:-1:1) #comparing [1 2 3] with [3 2 1]. It's true only for 2nd elements

3-element BitVector:
 0
 1
 0

In [7]:
.![true, false] #notice the order of `.` and `!`

2-element BitVector:
 0
 1

## Conditional evaluations

Implementation of  `if/elseif/else` is very standard and similar to other languages:

```
  if expr1 
      sth1
  elseif expr2
      sth2
  else 
      sth3
  end
```


* `if`  - do `sth1` if `expr1` is true
* `elseif` - do `sth2` if `expr2` is true and `expr1` is false 
* `else` - do `sth3` if both `expr1` and `expr2` are false

Example below:

In [8]:
if 2!=2
    println("asd")
elseif 3>4
    println("3>4")
elseif 5==10
    println("5==10")
else
    println("nothing")
end

nothing


# `?:` ternary operator

There is a ternary operator `?:` for short conditional evaluations. 
Its syntax is `cond ? sth1 : sth2`. The result of this operation conducted on scalar is: do `sth1` if `cond` is true and `sth2` otherwise.


In [9]:
x = 4
even_x = x%2==0 ? "x is even" : "x is odd"

"x is even"

`?:` can be nested, just like in an example below:

In [10]:
x = 4

x<5 ?  ( 
        x%2==0 ? println("x is smaller than 5 and is even") : #x<5 is true and x%2==0 is true
                 println("x is smaller than 5 and is odd")    #x<5 is true and x%2==0 is false
        ) :
         println("x is greater or equal to 5")                #x<5 is false and x%2==0 is not evaluated

x is smaller than 5 and is even


(elementwise-question)=
### Elementwise `?:`

(elementwise-question-example)=
```{note}
Suppose that we want to perform a following mapping for $x\in 1,2, \ldots, 10:$

$$ f(x) = \begin{cases}x^2+1, \text{if $x$ is odd,}\\
                       -x^2, \text{if $x$ is even.}\\ \end{cases}$$
```

Such a task can be done relatively easily in other languages.
For instance in `R` we get it by using `ifelse`: 

```
    result <- ifelse(x%%2==0, -x^2, x^2 + 1) #this is `R` code. It will not work in `Julia`
```

We might be tempted to make elementwise comparisons using `.?`.

In [11]:
x = collect( 1:10 );
result = ((x) .%2 .==0) .? -(x).^2 .: (x).^2.+1 

LoadError: syntax: space before "." not allowed in "((x .% 2) .== 0) ." at In[11]:2

As we can see, it will not work the way we would like. 
A solution to this problem is to use `map` function with an anonymous function:

In [115]:
map(arg -> arg%2==0 ? -arg^2 : arg^2+1 , x)

10-element Vector{Int64}:
    2
   -4
   10
  -16
   26
  -36
   50
  -64
   82
 -100

This works as intended.

(short-circuit)=
## Short-circuit evaluations

`Julia` exhibits lazy evaluation of formulas. 
This means that operations are computed only if they are needed.
This property has some particular consequences for two operators, `&&` and `||`.
Laziness of `Julia` together with necessary condition for `true` value in `&&` and sufficient condition for `false` in `||` can be used for conditional evaluations.
More precisely, the second (right-hand side) argument of `&&` is evaluated if and only if the first (left-hand side) argument is true.
Similarly, the second (right-hand side) argument of `||` is _not_ valuated if if the first (left-hand side) argument is _false._

As the second argument we can put some evaluations that we would like to conduct subject to the left-hand side argument.
Simple illustrations below:

In [126]:
(2+2==4) && println("(2+2==4) It's true");


(2+2==4) It's true


In [127]:
(2+2==5) && println("(2+2==5) It's true"); # the latter is not executed as (2+2==5) is false

In [2]:
(2+2==4) || println("(2+2==4) It's false"); # the latter is not executed as (2+2==4) is true

In [129]:
(2+2==5) || println("(2+2==5) It's false");

It's false
